In [1]:
from ultralytics import YOLO
import cv2
import os
import glob
import os
import glob
import random
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from ultralytics import YOLO
import torch
from pathlib import Path

In [3]:
import os
print(os.getcwd())


/Users/sahilbodkhe/Desktop/Experiment4/SYDE770


In [2]:
# Paths
BASE_DIR = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups"  # Change to your dataset path
IMAGE_DIR = os.path.join(BASE_DIR, "images")
LABEL_DIR = os.path.join(BASE_DIR, "data")
OUTPUT_DIR = os.path.join(BASE_DIR, "yolo_dataset")
MODEL_PATH = "yolov5"  # Change if using another YOLOv5 model

# Create output directories
os.makedirs(os.path.join(OUTPUT_DIR, "train", "images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "train", "labels"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val", "images"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "val", "labels"), exist_ok=True)

# Load images & labels
image_paths = glob.glob(os.path.join(IMAGE_DIR, "*"))
image_list = []
labels = []

for img_path in image_paths:
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(LABEL_DIR, base_name + ".txt")

    if os.path.exists(label_path) and os.path.getsize(label_path) > 0:
        with open(label_path, "r") as file:
            label_data = file.readlines()
            class_ids = [int(line.split()[0]) for line in label_data]
            label_class = 1 if 1 in class_ids else 0  # If a Tim Hortons cup is present, label as 1
    else:
        label_class = 0  # No annotation means it's a normal cup (background class)

    image_list.append((img_path, label_path))
    labels.append(label_class)

# Convert to DataFrame for processing
df = pd.DataFrame(image_list, columns=["image", "label"])
df["class"] = labels

# Print dataset summary
print(f"Total Images: {len(df)}")
print(f"Class Distribution: {df['class'].value_counts()}")


Total Images: 1669
Class Distribution: class
1    1026
0     643
Name: count, dtype: int64


In [4]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Create necessary directories and copy data (train/validation)
OUTPUT_DIR = "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset"

# Stratified K-Fold Cross Validation (Keep Class Balance)
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Select one fold as validation (80-20 Split)
for train_idx, val_idx in skf.split(df["image"], df["class"]):
    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]
    break  # We only need the first split

# Function to copy files
def copy_files(data, split):
    for _, row in data.iterrows():
        shutil.copy(row["image"], os.path.join(OUTPUT_DIR, split, "images", os.path.basename(row["image"])))

        if os.path.exists(row["label"]) and os.path.getsize(row["label"]) > 0:
            shutil.copy(row["label"], os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(row["label"])))
        else:
            # Create an empty label file if no annotation exists
            open(os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(row["label"])), 'w').close()

copy_files(train_data, "train")
copy_files(val_data, "val")

# Create YOLO dataset.yaml file
yaml_content = f"""
path: C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset
train: train/images
val: val/images
nc: 2
names: ["Normal Cup", "Tim Hortons Cup"]
"""

yaml_path = "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset/dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content)

# Train YOLOv5 (No MLflow integration in this cell)
command = r'python yolov5/train.py --imgsz 320 --batch-size 32 --epochs 5 --data "C:/Users/Chinmay Nagesh/Desktop/Yolo/SYDE770/Normal_cups/yolo_dataset/dataset.yaml" --weights yolov5s.pt --device cpu --freeze 10'
os.system(command)

print("Training completed! Model saved in YOLO output folder.")


Training completed! Model saved in YOLO output folder.


In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Define constants
OUTPUT_DIR = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset"


In [ ]:
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
VAL_DIR = os.path.join(OUTPUT_DIR, "val")

# Ensure necessary directories exist
def create_dirs():
    for split in ["train", "val"]:
        os.makedirs(os.path.join(OUTPUT_DIR, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, split, "labels"), exist_ok=True)



# Stratified K-Fold Split
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(df["image"], df["class"]):
    train_data = df.iloc[train_idx]
    val_data = df.iloc[val_idx]
    break  # Only using the first split

# Function to copy files
def copy_files(data, split):
    for _, row in data.iterrows():
        img_src = row["image"]
        label_src = row["label"]
        
        img_dst = os.path.join(OUTPUT_DIR, split, "images", os.path.basename(img_src))
        label_dst = os.path.join(OUTPUT_DIR, split, "labels", os.path.basename(label_src))
        
        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)
        else:
            print(f"Warning: Image file not found - {img_src}")
        
        if os.path.exists(label_src) and os.path.getsize(label_src) > 0:
            shutil.copy(label_src, label_dst)
        else:
            open(label_dst, 'w').close()  # Create empty label if missing

# Create required directories
create_dirs()
copy_files(train_data, "train")
copy_files(val_data, "val")

# Generate dataset.yaml file
yaml_content = f"""
path: {OUTPUT_DIR}
train: train/images
val: val/images
nc: 2
names: ["Normal Cup", "Tim Hortons Cup"]
"""

yaml_path = os.path.join(OUTPUT_DIR, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

# Train YOLOv5
command = (
    f'python3 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/yolov5/train.py --imgsz 320 --batch-size 32 --epochs 5 '
    f'--data "{yaml_path}" --weights yolov5s.pt --device mps --freeze 10'
)

exit_code = os.system(command)
if exit_code != 0:
    print("Error: YOLO training failed. Check the logs for details.")
else:
    print("Training completed! Model saved in YOLO output folder.")

In [8]:
import torch
torch.mps.empty_cache()

In [2]:
import torch
print(torch.backends.mps.is_available())


True


In [9]:
import torch
print(torch.backends.mps.is_available())  # True if Metal is available
print(torch.backends.mps.is_built())  # True if Metal is built into PyTorch


True
True


In [1]:
import os
import torch
from ultralytics import YOLO

# Define the output directory and YAML path
OUTPUT_DIR = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset"
yaml_path = os.path.join(OUTPUT_DIR, "dataset.yaml")

# Clear GPU cache before starting training
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
    device = "mps"
else:
    device = "cpu"  # Fallback to CPU

# Load a YOLOv8 model (e.g., yolov8s.pt)
model = YOLO("yolov8s.pt")

# Train the model
results = model.train(
    data=yaml_path,       # Path to the dataset YAML file
    epochs=10,            # Number of epochs
    batch=64,             # Batch size
    imgsz=416,            # Image size
    device=device,        # Use MPS (Metal Performance Shaders) if available, else CPU
    freeze=10             # Freeze the first 10 layers
)

# # Clear GPU cache after training
# torch.mps.empty_cache()

# if results.success:
#     print("Training completed! Model saved in YOLO output folder.")
# # else:
#     print("Error: YOLO training failed. Check the logs for details.")

New https://pypi.org/project/ultralytics/8.3.89 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.88 🚀 Python-3.11.9 torch-2.6.0 MPS (Apple M3)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/dataset.yaml, epochs=10, time=None, patience=100, batch=64, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

train: Scanning /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/labels.cache... 1335 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1335/1335 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/GenAI_Img_4_2.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/train/images/a_plain_red_coloured_tim_hortons_cup-19.jpeg: 1 duplicate labels removed



val: Scanning /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_cups/yolo_dataset/val/labels.cache... 334 images, 0 backgrounds, 0 corrupt: 100%|██████████| 334/334 [00:00<?, ?it/s]


Plotting labels to runs/detect/train10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
MLflow: logging run_id(9f41e2e316dd48ba93ad28a4e2e2ded2) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs/detect/train10
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      8.83G     0.6746     0.7344      1.078         76        416: 100%|██████████| 21/21 [11:28<00:00, 32.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:33<00:00, 31.04s/it]


                   all        334        503      0.955      0.965       0.98      0.912

10 epochs completed in 2.947 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 22.5MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics 8.3.88 🚀 Python-3.11.9 torch-2.6.0 MPS (Apple M3)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:23<00:00, 27.93s/it]


                   all        334        503      0.958      0.877      0.925      0.834
            Normal Cup        150        240      0.962      0.887      0.933      0.819
       Tim Hortons Cup        205        263      0.954      0.867      0.917      0.849
Speed: 8.9ms preprocess, 134.9ms inference, 0.0ms loss, 18.7ms postprocess per image
Results saved to runs/detect/train10
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


AttributeError: 'DetMetrics' object has no attribute 'success'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        names (dict of str): A dict of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        names (dict of str): A dict of strings that represents the names of the classes.
        box (Metric): An instance of the Metric class for storing the results of the detection metrics.

In [11]:
import ultralytics
print(ultralytics.__file__)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ultralytics/__init__.py


Keeping results ready for MLflow

In [15]:
import numpy
import ultralytics
print(numpy.__version__)  # Should print 2.1.1
print(ultralytics.__version__)  # Should match 8.3.88


2.1.1
8.3.88


In [5]:
import pandas as pd
import os

# Path to results.csv in latest exp folder
latest_exp = '/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/runs/detect/train10'
results_path = os.path.join(latest_exp, "results.csv")

# Check if results.csv exists
if os.path.exists(results_path):
    df = pd.read_csv(results_path)
    df.columns = df.columns.str.strip()

    # Extract the values from the last row of the DataFrame
    train_box_loss = df.iloc[-1]["train/box_loss"]
    train_dfl_loss = df.iloc[-1]["train/dfl_loss"]  # Corrected column name
    train_cls_loss = df.iloc[-1]["train/cls_loss"]
    val_box_loss = df.iloc[-1]["val/box_loss"]
    mAP_50 = df.iloc[-1]["metrics/mAP50(B)"]  # Corrected column name
    mAP_50_95 = df.iloc[-1]["metrics/mAP50-95(B)"]  # Corrected column name
    precision = df.iloc[-1]["metrics/precision(B)"]  # Corrected column name
    recall = df.iloc[-1]["metrics/recall(B)"]  # Corrected column name

    # Print or use the extracted values
    print(f"train_box_loss: {train_box_loss}")
    print(f"train_dfl_loss: {train_dfl_loss}")  # Updated variable name
    print(f"train_cls_loss: {train_cls_loss}")
    print(f"val_box_loss: {val_box_loss}")
    print(f"mAP_50: {mAP_50}")
    print(f"mAP_50_95: {mAP_50_95}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")

else:
    print(f"Error: {results_path} does not exist.")

train_box_loss: 0.67455
train_dfl_loss: 1.07821
train_cls_loss: 0.73439
val_box_loss: 0.36067
mAP_50: 0.98042
mAP_50_95: 0.91239
precision: 0.95465
recall: 0.96495


ML Flow section

In [5]:
import mlflow
import dagshub
import pandas as pd
import shutil
import os

# Initialize MLflow tracking
dagshub.init(repo_owner='chinmay-nagesh',
             repo_name='SYDE770-dagshub',
             mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow")
mlflow.set_experiment("YOLOv5 Training")

# Start MLflow run to track hyperparameters and metrics
with mlflow.start_run():
    # Log Hyperparameters
    mlflow.log_param("img_size", 320)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("epochs", 5)
    mlflow.log_param("weights", "yolov5s.pt")
    mlflow.log_param("device", "cpu")
    mlflow.log_param("freeze", 10)
    

    # Log Metrics
    mlflow.log_metric("train_box_loss", train_box_loss)
    mlflow.log_metric("train_obj_loss", train_obj_loss)
    mlflow.log_metric("train_cls_loss", train_cls_loss)
    mlflow.log_metric("val_box_loss", val_box_loss)
    mlflow.log_metric("mAP_50", mAP_50)
    mlflow.log_metric("mAP_50_95", mAP_50_95)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)

    # Log Model
    best_model_path = "yolov5/runs/train/exp/weights/best.pt"
    model_save_path = "yolov5_model"

    if os.path.exists(best_model_path):
        os.makedirs(model_save_path, exist_ok=True)
        shutil.copy(best_model_path, os.path.join(model_save_path, "best.pt"))

        # Log Model as an artifact in MLflow
        mlflow.log_artifact(os.path.join(model_save_path, "best.pt"), artifact_path="models/yolov5")

        # Check if model "yolov5" exists in MLflow model registry
        client = mlflow.tracking.MlflowClient()
        model_name = "yolov8"

        try:
            # Try registering a new version of the existing model
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov5/best.pt", model_name)
            print(f"New version of {model_name} registered in MLflow.")
        except Exception:
            # If model does not exist, create a new registered model
            mlflow.create_registered_model(model_name)
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov5/best.pt", model_name)
            print(f"Model {model_name} created and registered in MLflow.")

    print("Metrics and model logged to MLflow.")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=46bc746d-70c3-4d67-81e7-8da7e7067fce&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ec6b43d9757d4f5bac8e151aa46e6888dab2aeb74e1b2b4115c1bcab4ccf3dea




Accessing as SAHILBODKHE

Initialized MLflow to track repo "chinmay-nagesh/SYDE770-dagshub"

Repository chinmay-nagesh/SYDE770-dagshub initialized!

Metrics and model logged to MLflow.
🏃 View run fearless-cat-924 at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1/runs/020ca79ae9f24e208e4a8491445f4cf1
🧪 View experiment at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1


In [9]:
import mlflow
import dagshub
import pandas as pd
import shutil
import os

# Initialize MLflow tracking
dagshub.init(repo_owner='chinmay-nagesh',
             repo_name='SYDE770-dagshub',
             mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow")
mlflow.set_experiment("YOLOv5 Training")

# Start MLflow run to track hyperparameters and metrics
with mlflow.start_run():
    # Log Hyperparameters
    mlflow.log_param("img_size", 416)
    mlflow.log_param("batch_size", 64)
    mlflow.log_param("epochs", 10)
    mlflow.log_param("weights", "yolov8s.pt")
    mlflow.log_param("device", "mps")
    mlflow.log_param("freeze", 10)

    # Log Metrics (updated to match the correct column names from the CSV)
    mlflow.log_metric("train_box_loss", train_box_loss)
    mlflow.log_metric("train_dfl_loss", train_dfl_loss)  # Updated to match CSV column name
    mlflow.log_metric("train_cls_loss", train_cls_loss)
    mlflow.log_metric("val_box_loss", val_box_loss)
    mlflow.log_metric("mAP_50", mAP_50)
    mlflow.log_metric("mAP_50_95", mAP_50_95)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)  # Fixed typo: "recall" instead of "recall"

    # Log Model
    best_model_path = "runs/detect/train10/weights/best.pt"
    model_save_path = "yolov8_model"

    if os.path.exists(best_model_path):
        os.makedirs(model_save_path, exist_ok=True)
        shutil.copy(best_model_path, os.path.join(model_save_path, "best.pt"))

        # Log Model as an artifact in MLflow
        mlflow.log_artifact(os.path.join(model_save_path, "best.pt"), artifact_path="runs/mlflow/283939179369314934/9f41e2e316dd48ba93ad28a4e2e2ded2/artifacts")

        # Check if model "yolov5" exists in MLflow model registry
        client = mlflow.tracking.MlflowClient()
        model_name = "yolov8"  # Corrected model name to match YOLOv5

        try:
            # Try registering a new version of the existing model
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov8/best.pt", model_name)
            print(f"New version of {model_name} registered in MLflow.")
        except Exception as e:
            # If model does not exist, create a new registered model
            print(f"Error registering model: {e}")
            mlflow.create_registered_model(model_name)
            mlflow.register_model(f"runs:/{mlflow.active_run().info.run_id}/models/yolov8/best.pt", model_name)
            print(f"Model {model_name} created and registered in MLflow.")
    else:
        print(f"Error: Model file {best_model_path} does not exist.")

    print("Metrics and model logged to MLflow.")

Initialized MLflow to track repo "chinmay-nagesh/SYDE770-dagshub"

Repository chinmay-nagesh/SYDE770-dagshub initialized!

Successfully registered model 'yolov8'.
2025/03/13 01:47:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: yolov8, version 1
Created version '1' of model 'yolov8'.


New version of yolov8 registered in MLflow.
Metrics and model logged to MLflow.
🏃 View run angry-owl-868 at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1/runs/58aad525607346c8bec8e59feed148ff
🧪 View experiment at: https://dagshub.com/chinmay-nagesh/SYDE770-dagshub.mlflow/#/experiments/1


In [2]:
from ultralytics import YOLO
import os
import cv2

# Path to the trained model
model_path = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/runs/detect/train10/weights/best.pt"

# Path to the folder containing test images
test_images_folder = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Test_images"

# Path to save the output images with predictions
output_folder = "/Users/sahilbodkhe/Desktop/Experiment4/SYDE770/predictions"
os.makedirs(output_folder, exist_ok=True)

# Load the trained YOLOv8 model
model = YOLO(model_path)

# Run inference on all images in the Test_images folder
results = model.predict(source=test_images_folder, save=True, project=output_folder, name="predictions")

# # Function to display images with bounding boxes
# def display_images_with_boxes(results, output_folder):
#     for result in results:
#         # Get the image file name
#         image_name = os.path.basename(result.path)
#         # Load the output image with bounding boxes
#         output_image_path = os.path.join(output_folder, image_name)
#         output_image = cv2.imread(output_image_path)
        
#         # Display the image
#         cv2.imshow("Prediction", output_image)
#         cv2.waitKey(0)  # Wait for a key press to move to the next image
#         cv2.destroyAllWindows()

# # Call the function to display images
# display_images_with_boxes(results, output_folder)


image 1/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-10.jpeg: 416x416 4 Normal Cups, 63.5ms
image 2/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-2.jpeg: 416x416 3 Normal Cups, 49.6ms
image 3/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-3.jpeg: 416x416 2 Normal Cups, 46.9ms
image 4/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-4.jpeg: 416x416 4 Normal Cups, 50.5ms
image 5/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-5.jpeg: 416x416 4 Normal Cups, 43.8ms
image 6/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-6.jpeg: 416x416 3 Normal Cups, 48.3ms
image 7/10 /Users/sahilbodkhe/Desktop/Experiment4/SYDE770/Normal_Test/red_paper_cup_with_dark_shade-7.jpeg: 416x416 3 Normal Cups, 44.0ms
image 8/10 /Users/sahilbodkhe/De

In [ ]:
results = model.predict(source=test_images_folder, conf=0.5, save=True, project=output_folder, name="predictions")